In [ ]:
import cv2
import numpy as np

BP_ITERATIONS = 40
LABELS = 16
LAMBDA = 20
SMOOTHNESS_TRUNC = 2


class DIRECTION:
    LEFT = 0
    RIGHT = 1
    UP = 2
    DOWN = 3
    DATA = 4

class Pixel:
    def __init__(self):
        self.msg = np.zeros((5, LABELS), dtype=np.uint)
        self.best_assignment = 0

class MRF2D:
    def __init__(self, width=0, height=0):
        self.grid = []
        self.width = width
        self.height = height
		
def MAP(mrf):

    for node in mrf.grid:
        best_cost = float('inf')
        for label in range(LABELS):
            cost = (node.msg[DIRECTION.LEFT][label] + node.msg[DIRECTION.RIGHT][label] +
                    node.msg[DIRECTION.UP][label] + node.msg[DIRECTION.DOWN][label] +
                    node.msg[DIRECTION.DATA][label])

            if cost < best_cost:
                best_cost = cost
                node.best_assignment = label

    width = mrf.width
    height = mrf.height

    energy = 0

    for y in range(height):
        for x in range(width):
            current_label = mrf.grid[y*width + x].best_assignment

            # Data cost
            energy += mrf.grid[y*width + x].msg[DIRECTION.DATA][current_label]

            if x - 1 >= 0:
                energy += SmoothnessCost(current_label, mrf.grid[y*width + x - 1].best_assignment)
            if x + 1 < width:
                energy += SmoothnessCost(current_label, mrf.grid[y*width + x + 1].best_assignment)
            if y - 1 >= 0:
                energy += SmoothnessCost(current_label, mrf.grid[(y - 1)*width + x].best_assignment)
            if y + 1 < height:
                energy += SmoothnessCost(current_label, mrf.grid[(y + 1)*width + x].best_assignment)

    return energy



def InitDataCost(left_file, right_file, mrf):
    left = cv2.imread(left_file, 0)
    right = cv2.imread(right_file, 0)

    if left is None:
        print("Error reading left image")
        exit(1)

    if right is None:
        print("Error reading right image")
        exit(1)

    mrf.width = left.shape[1]
    mrf.height = left.shape[0]

    total = mrf.width * mrf.height

    mrf.grid = [Pixel() for _ in range(total)]

    # Add a border around the image
    border = LABELS

    for y in range(border, mrf.height-border):
        for x in range(border, mrf.width-border):
            for i in range(LABELS):
                mrf.grid[y*mrf.width+x].msg[DIRECTION.DATA][i] = DataCostStereo(left, right, x, y, i)


def DataCostStereo(left, right, x, y, label):
    wradius = 2  
    sum = 0
    for dy in range(-wradius, wradius+1):
        for dx in range(-wradius, wradius+1):
            a = left[y+dy, x+dx]
            b = right[y+dy, x+dx-label]
            sum += abs(a-b)

    avg = sum // ((wradius*2+1)*(wradius*2+1))  # average difference
    return avg


def SmoothnessCost(i, j):
    d = i - j
    return LAMBDA * min(abs(d), SMOOTHNESS_TRUNC)



def SendMsg(mrf, x, y, direction):
    new_msg = [float('inf')] * LABELS
    width = mrf.width

    for i in range(LABELS):
        min_val = float('inf')

        for j in range(LABELS):
            p = 0

            p += SmoothnessCost(i, j)
            p += mrf.grid[y * width + x].msg[DIRECTION.DATA][j]

            
            if direction != DIRECTION.LEFT: p += mrf.grid[y * width + x].msg[DIRECTION.LEFT][j]
            if direction != DIRECTION.RIGHT: p += mrf.grid[y * width + x].msg[DIRECTION.RIGHT][j]
            if direction != DIRECTION.UP: p += mrf.grid[y * width + x].msg[DIRECTION.UP][j]
            if direction != DIRECTION.DOWN: p += mrf.grid[y * width + x].msg[DIRECTION.DOWN][j]

            min_val = min(min_val, p)

        new_msg[i] = min_val

    for i in range(LABELS):
        if direction == DIRECTION.LEFT:
            mrf.grid[y * width + x - 1].msg[DIRECTION.RIGHT][i] = new_msg[i]
        elif direction == DIRECTION.RIGHT:
            mrf.grid[y * width + x + 1].msg[DIRECTION.LEFT][i] = new_msg[i]
        elif direction == DIRECTION.UP:
            mrf.grid[(y - 1) * width + x].msg[DIRECTION.DOWN][i] = new_msg[i]
        elif direction == DIRECTION.DOWN:
            mrf.grid[(y + 1) * width + x].msg[DIRECTION.UP][i] = new_msg[i]
        else:
            raise ValueError("Invalid direction")

def BP(mrf, direction):
    width = mrf.width
    height = mrf.height

    if direction == DIRECTION.RIGHT:
        for y in range(height):
            for x in range(width - 1):
                SendMsg(mrf, x, y, direction)

    elif direction == DIRECTION.LEFT:
        for y in range(height):
            for x in range(width - 1, 0, -1):
                SendMsg(mrf, x, y, direction)

    elif direction == DIRECTION.DOWN:
        for x in range(width):
            for y in range(height - 1):
                SendMsg(mrf, x, y, direction)

    elif direction == DIRECTION.UP:
        for x in range(width):
            for y in range(height - 1, 0, -1):
                SendMsg(mrf, x, y, direction)

    elif direction == DIRECTION.DATA:
        raise ValueError("Invalid direction")



def main():
    mrf = MRF2D()
    InitDataCost("Images/art_view1.png", "Images/art_view5.png", mrf)

    for i in range(BP_ITERATIONS):
        BP(mrf, DIRECTION.RIGHT)
        BP(mrf, DIRECTION.LEFT)
        BP(mrf, DIRECTION.UP)
        BP(mrf, DIRECTION.DOWN)

        energy = MAP(mrf)

        print(f"iteration {i+1}/{BP_ITERATIONS}, energy = {energy}")

    output = np.zeros((mrf.height, mrf.width), dtype=np.uint8)

    for y in range(LABELS, mrf.height - LABELS):
        for x in range(LABELS, mrf.width - LABELS):
           
            output[y, x] = mrf.grid[y * mrf.width + x].best_assignment * (256 // LABELS)

    cv2.namedWindow("main", cv2.WINDOW_AUTOSIZE)
    cv2.imshow("main", output)
    cv2.waitKey(0)

    print("Saving results to output.png")
    cv2.imwrite("output.png", output)

if __name__ == "__main__":
    main()
